## The Minimum Cost Prison Diet
**Written by: Team Margaret Reid**

## Introduction

Prison diets are known to be high in carbohydrates and sodium levels with little or no access to fresh produce. Often, this results in adverse health outcomes and can lead to increased spendings for medical needs. The goal of this project is to identify the minimum cost diet that satisfies nutritional needs for U.S. inmate population using available data on county, state, and federal prisons. We will come up with a minimum cost diet for the average prisoner and investigate whether or not state prisons are able to meet the suggested nutritional requirements based on their weekly menus.

### Setup

In [5]:
# Uncomment to run the line below when you are opening this notebook for the first time
# %pip install -r requirements.txt --upgrade

In [6]:
from Functions import solvercomplete, nutrition, nutrient_search
#for documentation on any function run "function?", ex nutrition?
import numpy as np
import pandas as pd
from eep153_tools.sheets import read_sheets

### Part 1: Setting constraints and general variables

In [8]:
age = 35 # Choose an appropriate age
sex = "M" # Set M for Male and F for Female
activity_level = "Active" # Options: Moderately Active, Sedentary, or Active

In [9]:
# time to make minimizing and maximizing constraints
bmin, bmax = nutrition(age, sex, activity_level)
print(f"Current minimizing constraints are: \n \n {bmin}")
print(f"Current maximum constraints are: \n \n {bmax}")

Current minimizing constraints are: 
 
 Nutrient
Energy            3000.0
Protein             56.0
Carbohydrate       130.0
Dietary Fiber       30.8
Linoleic Acid       17.0
Linolenic Acid       1.6
Calcium           1000.0
Iron                 8.0
Magnesium          420.0
Phosphorus         700.0
Potassium         4700.0
Zinc                11.0
Copper               0.9
Selenium            55.0
Vitamin A          900.0
Vitamin E           15.0
Vitamin D           15.0
Vitamin C           90.0
Thiamin              1.2
Riboflavin           1.3
Niacin              16.0
Vitamin B6           1.3
Vitamin B12          2.4
Choline            550.0
Vitamin K          120.0
Folate             400.0
Name: Male_31_50, dtype: float64
Current maximum constraints are: 
 
 Nutrient
Sodium    2300.0
Name: Male_31_50, dtype: float64


/home/jovyan/EEP-153-Project-2/Functions.py:40: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  bmin[0] = active_energy


#### A Note On Case Specific Constraint Editing
Feel free to add in other maximizing constraints if you think they are relevant. Just make sure to justify them

### Part 2: Determining recipe and nutrition set

**Nutrition set is what constrains the options, since we are using a left join (or merge) on nutrients and recipes. The Nutrition dataset provides nutritional values and the recipe set provides weights**

In [10]:
data_url = "https://docs.google.com/spreadsheets/d/1xqixhrAoDq9rWJf_FC3Y2eXdd010DTLPCS7JJMCfwP8/edit?usp=sharing"
recipes = read_sheets(data_url, sheet="recipes")
nutrients = read_sheets(data_url, sheet="nutrients")

**Now, drop any foods within ingredients that you want in order to constrain to the specific area of interest (ex. particular population or prison)**

Below is a search function. Feel free to use it to either view particular foods within the dataset or set cut = True if you want to simply cut out all food options in the final result that have the search term in it. The function searches by Ingredient Description.

In [20]:
#example use
nutrient_search('Cream', nutrients).head()
#other example uses
# nutrient_search(['dog', 'cinnamon'], nutrients) 
# -> this would return all entries containing dog or cinnamon
# nutrient_search(['dog', 'cinnamon'], nutrients, cut = True) 
# -> this would return all entries not containing dog or cinammon (acting as a filter)
# nutrient_search('cheese', nutrients)
# -> this would return all entries containing cheese in ingredient descriptions

,ingred_code,Ingredient description,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,Vitamin B12,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc
9,1012,"Cheese, cottage, creamed, large or small curd",0.064,0.070,0.263,0.778,0.047,0.345,0.714,0.105,...,0.43,0.0,0.046,0.0,0.1,0.08,0.0,0.0,79.79,0.40
10,1013,"Cheese, cottage, creamed, with fruit",0.070,0.061,0.401,1.159,0.137,0.438,0.899,0.088,...,0.53,0.0,0.068,1.4,0.0,0.04,0.0,0.4,79.64,0.33
11,1014,"Cheese, cottage, nonfat, uncreamed, dry, large...",0.009,0.006,0.020,0.070,0.012,0.050,0.067,0.003,...,0.46,0.0,0.016,0.0,0.0,0.01,0.0,0.0,81.01,0.47
14,1017,"Cheese, cream",0.868,1.004,3.275,9.405,0.651,3.365,7.776,1.212,...,0.22,0.0,0.056,0.0,0.0,0.86,0.0,2.1,52.62,0.50
40,1050,"Cream, fluid, light (coffee cream or table cream)",0.456,0.526,1.682,4.585,0.334,1.719,3.952,0.654,...,0.14,0.0,0.044,0.8,1.1,0.12,0.0,1.7,74.51,0.32


## Running the optimization
Here is a call to the optimization function. Make sure to pass in the updated nutrients you use to the function: *solvercomplete(sex, age,recipes, **updated nutrients**, bmin, bmax)*

In [22]:
nutrition?

Signature: nutrition(age, sex, activity_level=False)
Docstring:
    takes the following
    age: integer from 2 through 120
    sex: either "F" for female or "M" for male
    activity_level: "Sedentary", "Moderately
Active", or "Active".
    outputs:
    bmin: A series object indexed by nutrition category with the series being the values respective to the age range from input.
    to the age range from input
    A series object indexed by nutrition category with the series being the values respective 
    to the age range from input. Energy is changed based on activity level or baseline activity which is sedentary I believe.
    
File:      ~/EEP-153-Project-2/p2utilityfunctions.py
Type:      function

In [86]:
import time
start_time = time.time()
ages = [19, 35, 51]
output = []
for agesp in ages:
    bmin, bmax = nutrition(agesp, sex, "Active")
    cost = solvercomplete(sex, agesp, recipes, nutrients, bmin, bmax)
    output += [cost]
end_time = time.time()

/home/jovyan/EEP-153-Project-2/p2utilityfunctions.py:40: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  bmin[0] = active_energy


Cost of diet for M's of age 19 is $3.71 per day.

The diet will consist of (in 100s of grams or milliliters):
Milk, low fat (1%)                                          4.51
Soy milk, light, chocolate                                  7.87
Liverwurst                                                  0.03
Peanut butter, lower sugar                                  2.98
Tortilla, corn                                              0.36
Noodles, vegetable, cooked                                  0.65
Grits, NS as to regular, quick, or instant, no added fat    1.08
Fruit juice blend, citrus, 100% juice                       2.21
Corn oil                                                    0.07
Mayonnaise, regular                                         0.15
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               3000.0          3000.0
Protein          113.571278            

/home/jovyan/EEP-153-Project-2/p2utilityfunctions.py:40: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  bmin[0] = active_energy


Cost of diet for M's of age 35 is $3.68 per day.

The diet will consist of (in 100s of grams or milliliters):
Milk, low fat (1%)                                               4.15
Soy milk, light, chocolate                                       7.33
Chocolate milk, made from reduced sugar mix with low fat milk    0.94
Liverwurst                                                       0.04
Peanut butter, lower sugar                                       2.99
Noodles, vegetable, cooked                                       0.30
Grits, NS as to regular, quick, or instant, no added fat         1.47
Fruit juice blend, citrus, 100% juice                            2.16
Mayonnaise, regular                                              0.37
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               3000.0          3000.0
Protein          111.547992            56.0
Carbohydrate   

/home/jovyan/EEP-153-Project-2/p2utilityfunctions.py:40: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  bmin[0] = active_energy


Cost of diet for M's of age 51 is $3.62 per day.

The diet will consist of (in 100s of grams or milliliters):
Milk, low fat (1%)                                               2.70
Soy milk, light, chocolate                                       5.19
Chocolate milk, made from reduced sugar mix with low fat milk    4.69
Liverwurst                                                       0.04
Peanut butter, lower sugar                                       2.69
Noodles, vegetable, cooked                                       0.39
Grits, NS as to regular, quick, or instant, no added fat         0.37
Fruit juice blend, citrus, 100% juice                            1.94
Mayonnaise, regular                                              0.36
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2800.0          2800.0
Protein          107.772569            56.0
Carbohydrate   

In [17]:
print(f"Execution time: {end_time - start_time:.2f} seconds")
print(ages)
print(output)

Execution time: 42.87 seconds
[19, 35, 51]
['4.95', '4.88', '4.76']


In [28]:
import time

start_time = time.time()
solvercomplete(sex, age, recipes, nutrients, bmin, bmax)
end_time = time.time()

print(f"Execution time: {end_time - start_time:.2f} seconds")

Cost of diet for M's of age 35 is $5.02 per day.

The diet will consist of (in 100s of grams or milliliters):
Milk, low fat (1%)                                     25.42
Whole wheat cereal, cooked, no added fat               24.91
Cereal (Post Honey Bunches of Oats with Almonds)        0.01
Mayonnaise, regular                                     0.93
Fruit juice drink, with high vitamin C, light           1.24
Water, bottled, flavored (Capri Sun Roarin' Waters)     3.95
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2800.0          2800.0
Protein          119.546426            56.0
Carbohydrate     372.121232           130.0
Dietary Fiber          28.0            28.0
Linoleic Acid     39.869637            14.0
Linolenic Acid     5.389177             1.6
Calcium         3381.501695          1200.0
Iron              11.407889             8.0
Magnesium      

In [85]:
solvercomplete(sex, age,recipes, nutrients, bmin, bmax)

Cost of diet for M's of age 35 is $3.68 per day.

The diet will consist of (in 100s of grams or milliliters):
Milk, low fat (1%)                                               4.15
Soy milk, light, chocolate                                       7.33
Chocolate milk, made from reduced sugar mix with low fat milk    0.94
Liverwurst                                                       0.04
Peanut butter, lower sugar                                       2.99
Noodles, vegetable, cooked                                       0.30
Grits, NS as to regular, quick, or instant, no added fat         1.47
Fruit juice blend, citrus, 100% juice                            2.16
Mayonnaise, regular                                              0.37
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               3000.0          3000.0
Protein          111.547992            56.0
Carbohydrate   

'3.68'

## Analysis on State Prison Menus

The lists of food items in this section are adapted from state prison weekly menus in California, Pennyslvania, and Florida. Using these food items to calculate a min-cost diet will tell us whether the menus are able to satisfy nutritional needs and if so, whether or not it is at an affordable price.

In [30]:
cal_food_items = [
    "Apple Juice", "Orange Juice", "Oatmeal", "Creamed Beef", "Biscuit", "Diced Potatoes", 
    "Ketchup", "1% Milk", "Coffee", "Pinto Bean", "Gravy", "Milk", "Peanut Butter", "Jelly", 
    "Almond", "Oatmeal Cookie", "Apple", "Orange", "Banana", "Lettuce", "Chicken Vegetable Stew", 
    "Corn Bread", "Turkey Hot Dog", "Vegetable Stew", "Low Fat Yogurt", "Bran Cake", "Bologna", 
    "Mayonnaise", "Mustard", "Whole Wheat Bread", "Corn Chips", "Coleslaw", "Hot Link", 
    "New Orleans Red Bean", "Red Bean", "Rice", "Cake with Icing", "Chocolate Milk", "Cereal", "Egg", 
    "Onions", "Bell Peppers", "Hummus", "Pretzel", "Sunflower Seed", "Bean & Cheese Burrito", 
    "Breakfast Burritos", "Chicken Chorizo", "Taco Sauce", "Tortillas", "Shredded Cheese", 
    "Roasted Chicken Lunchmeat", "Peanut Butter Cracker", "Gold Fish", "Chicken Taco Salad", 
    "Corn Tortilla", "Cheese Pizza", "Italian White Beans", "Pancake", "Syrup", "Beef Ravioli", 
    "Tuscan Style Bean", "Sugar Free Beverage", "Carrots", "Broccoli", "Chicken"
]

In [31]:
cal_nutrients = nutrient_search(cal_food_items, nutrients, cut = False)
solvercomplete(sex, age, recipes, cal_nutrients, bmin, bmax)

Cost of diet for M's of age 35 is $3.59 per day.

The diet will consist of (in 100s of grams or milliliters):
Milk, low fat (1%)                             7.68
Egg, yolk only, raw                            0.30
Peanut butter, reduced fat                     1.30
Chinese pancake                                5.15
Cereal, toasted oat                            0.58
Banana, raw                                    5.02
Mustard greens, fresh, cooked, no added fat    0.31
Mayonnaise, regular                            0.22
Fruit juice drink, added calcium (Sunny D)     0.52
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               3000.0          3000.0
Protein            97.82343            56.0
Carbohydrate     486.053327           130.0
Dietary Fiber          30.8            30.8
Linoleic Acid     23.428104            17.0
Linolenic Acid          1.6             1.6
C

'3.59'

In [16]:
pen_food_items = [
    "Grit", "Egg", "Toast", "Margarine", "Milk", "Coffee", "Sugar", 
    "Apple", "Orange", "Banana", "Meatloaf Patty", "Soy Loaf", "Gravy", "Whipped Potato", 
    "Wax Bean", "Bread", "Grilled Cheese Sandwich", "Peanut Butter", "Jelly", "Tomato Soup", 
    "Pasta", "Oatmeal", "Breakfast Pastry", "Hamburger", "Bean Burger", "Potato", "Corn", 
    "Ketchup", "Mustard", "Mayonnaise", "Coleslaw", "Rice", "Cornbread", "Frankfurters", 
    "Macaroni Salad", "Baked Bean", "Broccoli", "Jello", "Hard Cooked Eggs", 
    "Poultry Cacciatore", "Soy Cacciatore", "Pasta and Bean Casserole", "Lettuce", 
    "Salad Dressing", "Cream of Wheat", "Breaded Chicken Patty", "Pinto Beans", 
    "Spinach", "Noodle", "Carrot", "Meat Sauce Pasta Casserole", 
    "Soy Pasta Sauce Pasta Casserole", "Breaded Fish", "Creamed Soy Crumble", 
    "Creamed Beef", "Bean Paste", "Cocktail Sauce", "Tartar", "Vegetable Soup", 
    "Pancake", "Syrup", "Pudding", "Cheese Steak", "Tofu Oat Burger", "Flour Tortilla"
]

In [17]:
pen_nutrients = nutrient_search(pen_food_items, nutrients, cut = False)
solvercomplete(sex, age, recipes, pen_nutrients, bmin, bmax)

Cost of diet for M's of age 35 is $2.99 per day.

The diet will consist of (in 100s of grams or milliliters):
Hot chocolate / Cocoa, made with no sugar added dry mix and low fat milk    5.12
Egg, yolk only, raw                                                         0.36
Peanut butter, lower sugar                                                  0.21
Tortilla, corn                                                              2.37
Rice, white, cooked, made with margarine                                    5.11
Potato, roasted, from fresh, peel eaten, no added fat                       3.26
Mustard greens, fresh, cooked, no added fat                                 0.84
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               3000.0          3000.0
Protein           66.096445            56.0
Carbohydrate     325.645173           130.0
Dietary Fiber          30.8       

'2.99'

In [24]:
flo_food_items = [
    "Pancake", "Fresh Fruit", "Syrup", "Biscuit", "Jelly", "Hashbrown Potato", 
    "Sugar Packet", "Meatloaf Patty", "Mashed Potato", "Cabbage", "Sweet Pea", 
    "Dried Beans", "Meatball", "Italian Red Sauce", "Sausage", "Buttery Oatmeal", 
    "Potato Salad", "Fudge Brownies", "Baked Bean", "Rice", "Green Beans", 
    "Broccoli", "Cookie", "Tortilla", "Lettuce", "Pasta Salad", "Herbed Chicken Leg", 
    "Bran Flaked Cereal", "Carrot", "Scrambled Egg", "Vegetarian Cheese", "Oatmeal", 
    "Toast", "Rice", "Pasta", "Bread", "Orange", "Banana", "Apple", "Margarine", 
    "Tea", "Coffee", "Corn", "Sloppy Joe", "Taco Skillet", "Dressing", "Gravy", 
    "Grits", "Turkey", "Country Patty", "Pasta", "Buffalo Chicken Patty", "Cheese", 
    "Coleslaw", "Southern BBQ", "Bran Muffin", "Blueberry Muffin", "Potato"
]

In [25]:
flo_nutrients = nutrient_search(flo_food_items, nutrients, cut = False)
solvercomplete(sex, age,recipes, flo_nutrients, bmin, bmax)

Cost of diet for M's of age 35 is $4.36 per day.

The diet will consist of (in 100s of grams or milliliters):
Liverwurst                                               1.56
Tortilla, corn                                           0.51
Pasta, whole grain, cooked                               1.63
Rice, white, cooked, made with margarine                 2.04
Cereal, rice flakes                                      0.21
Oatmeal cereal, baby food, dry, instant                  0.68
Potato, roasted, from fresh, peel eaten, no added fat    6.81
Cabbage, green, raw                                      0.30
Corn oil                                                 0.59
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               3000.0          3000.0
Protein           64.848741            56.0
Carbohydrate     325.214302           130.0
Dietary Fiber          30.8            30.8

'4.36'

In [33]:
# This returns a min-cost diet by using food items from across the 3 state prison menus.

combined_nutrients = nutrient_search(cal_food_items + pen_food_items + flo_food_items, nutrients)
solvercomplete(sex, age,recipes, combined_nutrients, bmin, bmax)

Cost of diet for M's of age 35 is $2.94 per day.

The diet will consist of (in 100s of grams or milliliters):
Hot chocolate / Cocoa, made with no sugar added dry mix and low fat milk    3.42
Egg, yolk only, raw                                                         0.37
Peanut butter, lower sugar                                                  0.90
Rice, white, cooked, made with margarine                                    5.34
Cereal (Kellogg's All-Bran)                                                 0.35
Potato, roasted, from fresh, peel eaten, no added fat                       3.60
Mustard greens, fresh, cooked, no added fat                                 0.69
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               3000.0          3000.0
Protein           67.673624            56.0
Carbohydrate     248.200004           130.0
Dietary Fiber          30.8       

'2.94'